## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.

## 이 스크립트의 목적:
- LangGraph를 사용하여 데이터의 흐름을 그래프로 정의
- 텍스트를 벡터로 변환하고 이를 Qdrant에 저장
- 저장된 벡터를 기반으로 유사도 검색 수행

## 왜 필요한가?
- LangGraph를 사용하면 데이터 처리를 직관적으로 구성할 수 있음
- 벡터 변환 및 데이터 저장 과정을 워크플로우로 관리 가능

## 주요 개념:
- **LangGraph**: 데이터 흐름을 그래프로 표현하는 Python 라이브러리
- **Qdrant**: 대규모 벡터 데이터를 저장하고 검색하는 고속 데이터베이스
- **워크플로우 자동화**: 데이터 변환 및 저장을 구조적으로 정의하는 기법

In [1]:
# 의존성 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
from langgraph.graph import StateGraph, Graph, START, END
from typing import TypedDict, Annotated
import operator
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, VectorParams

In [2]:
# Qdrant 클라이언트 설정 (아래에서 오류가 발생한다면 qdrant 서버를 띄워주세요)
qdrant_client = QdrantClient(host="localhost", port=6333)

In [3]:
collection_name = "ex3_sample_vectors"

# 컬렉션 존재 여부 확인 후 생성
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384는 임베딩 차원 수
    )


In [4]:
# Sentence Transformers 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
# LangGraph에서 사용할 상태 타입 정의
class VectorState(TypedDict):
    text: str
    vector: Annotated[list[float], operator.iadd]  # 벡터 리스트를 병합하기 위해 iadd 사용

# 벡터 생성 함수
def generate_vector(state: VectorState) -> VectorState:
    vector = embeddings.embed_query(state["text"])
    print(f"'{state['text']}' → 벡터 변환 완료")
    return {"text": state["text"], "vector": vector}

# Qdrant에 벡터 저장 함수
def store_in_qdrant(state: VectorState) -> VectorState:
    qdrant_client.upsert(
        collection_name=collection_name,
        points=[models.PointStruct(id=hash(state["text"]), vector=state["vector"], payload={"text": state["text"]})]
    )
    print(f"'{state['text']}' 벡터가 Qdrant에 저장됨")
    return state

In [6]:
# 그래프 빌더 생성 및 노드 추가
graph_builder = StateGraph(VectorState)
graph_builder.add_node("generate_vector", generate_vector)
graph_builder.add_node("store_in_qdrant", store_in_qdrant)
graph_builder.add_edge(START, "generate_vector")
graph_builder.add_edge("generate_vector", "store_in_qdrant")
graph_builder.add_edge("store_in_qdrant", END)

In [7]:
# 그래프 컴파일 및 실행
graph = graph_builder.compile()
graph.invoke({"text": "LangGraph enables structured workflows!"})


'LangGraph enables structured workflows!' → 벡터 변환 완료
'LangGraph enables structured workflows!' 벡터가 Qdrant에 저장됨


{'text': 'LangGraph enables structured workflows!',
 'vector': [-0.04834276810288429,
  0.04105396568775177,
  0.028567897155880928,
  -0.06952270865440369,
  -0.026820452883839607,
  -0.042761996388435364,
  0.018161965534090996,
  0.0005122683360241354,
  0.049716368317604065,
  -0.01770610362291336,
  -0.05630359798669815,
  -0.031377166509628296,
  0.010021314956247807,
  0.05707448720932007,
  0.0499667264521122,
  0.030458908528089523,
  -0.04763372614979744,
  0.05919064208865166,
  -0.06125058978796005,
  -0.13379372656345367,
  -0.02877732552587986,
  0.028096795082092285,
  -0.010645118542015553,
  0.0244012288749218,
  0.028748055920004845,
  0.041076574474573135,
  -0.0771876648068428,
  -0.06830567866563797,
  0.12701666355133057,
  -0.006993924733251333,
  0.008960955776274204,
  0.12738937139511108,
  0.018098758533596992,
  0.11188110709190369,
  -0.007893718779087067,
  0.09159675985574722,
  -0.005207295063883066,
  -0.010932491160929203,
  -0.024614164605736732,
  -0